In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import pickle
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

desired_frames = 1
desired_features = 2816

In [2]:
labels = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


In [4]:
scenes = list(pd.unique(labels["SCENE"]))

In [5]:
len(scenes)

1202

In [6]:
file = open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/MultiModal/Early Fusion/Data/text_embed_pca_final.pickle', 'rb')
text_data = pickle.load(file)
file.close()

In [7]:
file = open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/MultiModal/Early Fusion/Data/visual_embed_padded_final.pickle', 'rb')
video_data = pickle.load(file)
file.close()

In [8]:
tv_data = {}
for scene in scenes:
    tv_data[scene] = np.concatenate((text_data[scene], video_data[scene]), axis=1)


In [9]:
tv_data['1_10004'].shape

(18, 2816)

In [10]:
def get_model_data(tv_data):
    model_data = pd.DataFrame(columns=['tv_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
#         audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'tv_feature': tv_data[row['SCENE']], 
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [11]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data
   

In [12]:
class GRUTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'padded_tv_feature']
            a=np.empty((18,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'padded_tv_feature']
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label

    def __getindexlist__(self):
        return list(self.data.index.values)
    
class GRUNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
class GRUNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
#         weight = next(self.parameters()).data
#         hidden = weight.new(self.n_layers, batch_size, 
#                             self.hidden_dim).zero_()
        return torch.zeros(self.n_layers, batch_size, self.hidden_dim)
#         return hidden
    
def evaluateGRU(gru, review, size):
    hidden = gru.init_hidden(size)
    output, hidden = gru(review, hidden)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(gru, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateGRU(gru, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals
    
hidden_size = 18
output_size = 2
input_size_sd = 2817
n_layers = 1

In [13]:
warnings.filterwarnings("ignore")

model_data = get_model_data(tv_data)
# Label Encode Speaker
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,tv_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[-5.775393, -0.55747294, -5.9735856, 0.882094...",0.0,NONE,SHELDON,25
1,"[[-9.726368, -4.6866755, -11.944455, -6.425826...",0.0,NONE,PENNY,15
2,"[[-14.0798025, 4.135473, -8.056498, -4.865928,...",0.0,NONE,RAJ,21
3,"[[-5.3839507, 2.0448134, -13.128815, -6.819113...",1.0,PRO,HOWARD,7
4,"[[-3.3831522, 5.3072224, -5.980673, -1.8846271...",0.0,NONE,SHELDON,25


In [14]:
desired_length = 18

train_data, test_data = get_train_test_split(model_data, ['tv_feature', 'speaker_encode'], 'sarcasm', 'sarcasm')
gru_train = train_data.copy()
gru_test = test_data.copy()
gru_train.reset_index(drop=True, inplace = True)
gru_test.reset_index(drop=True, inplace = True)

        
gru_train['padded_tv_feature'] = gru_train.loc[:, 'tv_feature']
gru_test['padded_tv_feature'] = gru_test.loc[:, 'tv_feature']

gru_train["sarcasm"] = gru_train["sarcasm"].astype('int').to_numpy()
gru_test["sarcasm"] = gru_test["sarcasm"].astype('int').to_numpy()

Train:  (961, 2) (961,) Test:  ((241, 2), (241,))


### Speaker Dependent

In [15]:
gru_train_tensor = GRUTensorDataset(gru_train[['padded_tv_feature', 'speaker_encode','sarcasm']], True)
gru_test_tensor = GRUTensorDataset(gru_test[['padded_tv_feature', 'speaker_encode','sarcasm']], True)

num_of_workers = 0
batch_size = 32
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices),
    drop_last=True
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices),
    drop_last=True
)

In [16]:
gru = GRUNetSD(input_size_sd, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSD(
  (gru): GRU(2817, 18, batch_first=True)
  (fc): Linear(in_features=18, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [17]:
n_epochs = 41
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "grumodel.pt")
#         test_min_loss = test_loss
    if(epoch%5 == 0):
        print("Epoch: " + str(epoch))
        test_loader = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader, gru_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)
        test_loader = torch.utils.data.DataLoader(gru_test_tensor, batch_size=batch_size, sampler=SubsetRandomSampler(test_indices))

Epoch: 0
              precision    recall  f1-score     support
0              0.566879  0.735537  0.640288  121.000000
1              0.619048  0.433333  0.509804  120.000000
accuracy       0.585062  0.585062  0.585062    0.585062
macro avg      0.592963  0.584435  0.575046  241.000000
weighted avg   0.592855  0.585062  0.575317  241.000000
Epoch: 5
              precision    recall  f1-score     support
0              0.606299  0.636364  0.620968  121.000000
1              0.614035  0.583333  0.598291  120.000000
accuracy       0.609959  0.609959  0.609959    0.609959
macro avg      0.610167  0.609848  0.609629  241.000000
weighted avg   0.610151  0.609959  0.609676  241.000000
Epoch: 10
              precision    recall  f1-score     support
0              0.610169  0.595041  0.602510  121.000000
1              0.601626  0.616667  0.609053  120.000000
accuracy       0.605809  0.605809  0.605809    0.605809
macro avg      0.605898  0.605854  0.605782  241.000000
weighted avg   0.605